In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, Math, Latex

In [2]:
from sympy import *
from sympy.vector import CoordSys3D
import numpy as np
from functools import reduce
from sympy.abc import lamda as lmd

In [3]:
def print_eq(eq, *exprs):
    if type(eq) == str:
        eq = Symbol(eq)
    out = '$$ {} ' + '= {}' * len(exprs) + '$$'
    exprs=list(exprs)
    for i in range(len(exprs)):
        if isinstance(exprs[i], (np.ndarray,)):
            exprs[i] = pmatrix(exprs[i])
        elif type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    display(Latex(out.format(latex(eq), *exprs)))
    
def to_s(i):
    return Symbol(str(i))

def pmatrix(array):
    if type(array) != Matrix:
        array = Matrix(array)
    return latex(array)[6:-7].replace('matrix', 'pmatrix') + ''

In [32]:
# исходники для первого задания
def task_vectors(a, f, f_solving):
    bar_a = Symbol(r'\bar a')
    bar_b = Symbol(r'\bar b')
    
    a = [np.array(el) for el in a]
    
    n = len(a)
    b = [
        a[0]
    ]

    print_eq(f'{bar_b}_1', f'{bar_a}_1')

    for i in range(1, n):
        b_gen_view = f'{bar_b}_{i + 1}={bar_a}_{i + 1}'
        b_solved_view = f'{bar_b}_{i + 1}={pmatrix(a[i])}'
        p1 = a[i]
        b_n = p1


        for j in range(1, i + 1):
            b_gen_view += f'-\\frac{{({bar_a}_{i + 1};{bar_b}_{j})}}{{({bar_b}_{j};{bar_b}_{j})}}{bar_b}_{j}'
        print_eq(b_gen_view)    

        for j in range(1, i + 1):
            err = None
            
            
            print_eq(f'({bar_a}_{{{i + 1}}};{bar_b}_{j})',f_solving(a[i], b[j - 1]), latex(f(a[i], b[j - 1])))
            
            p_n = Rational(
                f(a[i], b[j - 1]),
                f(b[j - 1], b[j - 1])
            )
            if p_n == nan:
                err = ZeroDivisionError('Получаем деление на 0')
                break

            b_solved_view += '-' + latex(p_n)
            b_solved_view += r'\cdot' + pmatrix(b[j - 1])
            p_n = p_n * b[j - 1]
            b_n = b_n - p_n
        
        print_eq(f'({bar_b}_{{{j}}};{bar_b}_{j})', f_solving(b[j - 1], b[j - 1]), latex(f(b[j - 1], b[j - 1])))

        b_solved_view += '=' + pmatrix(b_n)

        if not err:
            nok = 1
            for el in b_n:
                nok = np.gcd(nok, el)
            b_n = b_n / nok
    
            nod = reduce(np.gcd, b_n)
            b_n = b_n / (1 if not nod else nod)

        b.append(b_n)

        print_eq(b_solved_view)
        print_eq(f'{bar_b}_{i + 1}', (b_n))

        print('Проверка:')
        for j in range(i):
            print_eq(f'({bar_b}_{{{j + 1}}};{bar_b}_{i + 1})', f(b[j], b[i]))
    
    b_b_sqrts = []
    
    for i in range(len(b)):
        b_b_sqrt = sqrt(f(b[i], b[i]))
        if b_b_sqrt:
            b_b_sqrts.append(b_b_sqrt)
            print_eq(f'|{bar_b}_{{{i + 1}}}|', b_b_sqrt)

    for i in range(len(b_b_sqrts)):
        h = b[i] / b_b_sqrts[i]
        print_eq(f'\\bar h_{{{i + 1}}}', pmatrix(h))

In [50]:
# исходники для второго-третьего задания
# exprs - выражения, f - функция, f - решение функции, task_symbol - символ в задании
def task(exprs, f, f_solving, task_symbol):
    
    for i in range(len(exprs)):
        expr = exprs[i]
        print_eq(latex(task_symbol) + f"_{i + 1}", expr)
    
    b = [
        exprs[0]
    ]
    
    n = len(exprs)
    
    print_eq(f'b_1', f'{task_symbol}_1')

    for i in range(1, n):
        b_gen_view = f'b_{i + 1}={task_symbol}_{i + 1}'
        b_solved_view = f'b_{i + 1}=({latex(exprs[i])})'
        p1 = exprs[i]
        b_n = p1


        for j in range(1, i + 1):
            b_gen_view += f'-\\frac{{({task_symbol}_{i + 1};b_{j})}}{{(b_{j};b_{j})}}b_{j}'
        print_eq(b_gen_view)    

        for j in range(1, i + 1):
            
            print_eq(f'({task_symbol}_{{{i + 1}}};b_{j})', f_solving(exprs[i], b[j - 1]), f(exprs[i], b[j - 1]))
            p_n = Rational(
                f(exprs[i], b[j - 1]),
                f(b[j - 1], b[j - 1])
            )

            b_solved_view += '-' + f"({latex(p_n)})"
            b_solved_view += r'\cdot ' + f'({latex(b[j - 1])})'
            p_n = p_n * b[j - 1]
            b_n = b_n - p_n
            
        print_eq(f'(b_{{{j}}};b_{j})', f_solving(b[j - 1], b[j - 1]), f(exprs[j - 1], b[j - 1]))
        
        b_solved_view += '=' + latex(b_n)

        b.append(b_n)

        print_eq(b_solved_view)

        print('Проверка:')
        for j in range(i):
            print_eq(f'(b_{{{j + 1}}};b_{i + 1})', f(b[j], b[i]))
        
    b_b_sqrts = []
    for i in range(len(b)):
        b_b_sqrt = sqrt(f(b[i], b[i]))
        b_b_sqrts.append(b_b_sqrt)
        print_eq(f'|b_{{{i + 1}}}|', b_b_sqrt)

    for i in range(len(b)):
        h = b[i] / b_b_sqrts[i]
        print_eq(f'h_{{{i + 1}}}', expand(h))
        
f = Symbol('f')
g = Symbol('g')

t = Symbol('t')

# Задание 1

###### Настройка для второго задания

In [6]:
f_1 = np.dot

def f_1_solving(x, y):
    return "+".join([f"{e1}\cdot {e2}" for e1, e2 in zip(x, y)])


## Вставьте сюда свои значения

In [57]:
a = [
    [2, 2, 3, 1],
    [2, 5, 1, 5],
    [1, 2, 1, 5],
    [1, -1, 3, 1]
]

task_vectors(a, f_1, f_1_solving)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Задание 2

###### Настройка для второго задания

In [35]:
integral_from = 1
integral_to = -1

In [36]:
def f_2(x, y): # результирующая функция
    return Integral(x * y, (t, integral_to, integral_from)).doit()

def f_2_solving(x, y): # возвращает latex решения
    result = ""
    result += latex(Integral(x * y, (t, integral_to, integral_from))) + "="
    result += latex(Integral(x * y, (t,)).doit()) + f"\\bigg |_{{{integral_to}}}^{{{integral_from}}}"
    return result

In [37]:
print_eq(f'(f(x);g(x))=\int_{{{integral_to}}}^{{{integral_from}}} (f(x);g(x)) \,dx')

<IPython.core.display.Latex object>

## a)

### Вставьте свои значения сюда

In [59]:
exprs_2a = [
    -3 + 1 * t + 2 * t ** 2,
    -2 + 0 * t + 1 * t ** 2,
    3 + 0 * t - 1 * t ** 2,
]

task(exprs_2a, f_2, f_2_solving, f)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## б)

### Вставьте свои значения сюда

In [60]:
exprs_2b = [
    -3 - 2 * t - 7 * t ** 2,
    -2 - 1 * t - 3 * t ** 2,
    4 + 3 * t + 10 * t ** 2,
]
task(exprs_2b, f_2, f_2_solving, g)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Задание 3

###### Настройка для задания

In [61]:
def f_3(x, y): # результирующая функция
    result = 0
    result += x.subs({t: -1}) * y.subs({t: -1})
    result += x.subs({t: 0}) * y.subs({t: 0})
    result += x.subs({t: 1}) * y.subs({t: 1})
    return result

def f_3_solving(x, y): # возвращает latex решения
    result = ""
    result += latex(x.subs({t: -1})) + r"\cdot" + latex(y.subs({t: -1}))
    result += "+" + latex(x.subs({t: 0})) + r"\cdot" + latex(y.subs({t: 0}))
    result += "+" + latex(x.subs({t: 1})) + r"\cdot" + latex(y.subs({t: 1}))
    return result

In [62]:
print_eq(f'(f(x);g(x))=f(-1) \cdot g(-1) + f(0) \cdot g(0) + f(1) \cdot g(1)')

<IPython.core.display.Latex object>

## a)

### Вставьте свои значения сюда

In [63]:
exprs_3a = [
    -3 + 1 * t + 2 * t ** 2,
    -2 + 0 * t + 1 * t ** 2,
    3 + 0 * t - 1 * t ** 2,
]

task(exprs_3a, f_3, f_3_solving, f)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## б)

### Вставьте свои значения сюда

In [65]:
exprs_3b = [
    -3 - 2 * t - 7 * t ** 2,
    -2 - 1 * t - 3 * t ** 2,
    4 + 3 * t + 10 * t ** 2,
]

task(exprs_3b, f_3, f_3_solving, g)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>